<h1>Creating Custom Cost Functions</h1>

In this tutorial, we show how to create a custom cost function that might be needed for an application. While we can always use the `AutoDiffCostFunction` by simply writing an error function, it is often more efficient for compute-intensive applications to derive a new `CostFunction` subclass and use closed-form Jacobians. 

We will show how to write a custom `VectorDifference` cost function in this tutorial. This cost function provides the difference between two `Vector`s as the error. 

Note: `VectorDifference` is a simplified version of the `Difference` cost function already provided in the Theseus library, and shown in Tutorial 0. `Difference` can be used on any LieGroup, while `VectorDifference` can only be used on Vectors.

<h2>Initialization</h2> 

Any `CostFunction` subclass should be initialized with a `CostWeight` and all arguments needed to compute the cost function. In this example, we set up `__init__` function for `VectorDifference` to require as input the two `Vector`s whose difference we wish to compute: the `Vector` to be optimized, `var`, and the `Vector` that is the reference for comparison, `target`. 

In addition, the `__init__` function also needs to register the optimization variables and all the auxiliary variables. In this example, optimization variable `var` is registered with `register_optim_vars`. The other input necessary to evaluate the cost, `target` is registered with `register_aux_vars`. This is required for the nonlinear optimizers to work correctly: these functions register the optimization and auxiliary variables into internal lists, and then are easily used by the relevant `Objective` to add them, ensure no name collisions, and to update them with new values.

The `CostWeight` is used to weight the errors and jacobians, and is required by every `CostFunction` sub-class (the error and jacobian weighting functions are inherited from the parent `CostFunction` class.)

In [1]:
from typing import List, Optional, Tuple
import theseus as th

class VectorDifference(th.CostFunction):
    def __init__(
        self,
        cost_weight: th.CostWeight,
        var: th.Vector,
        target: th.Vector,
        name: Optional[str] = None,
    ):
        super().__init__(cost_weight, name=name) 

        # add checks to ensure the input arguments are of the same class and dof:
        if not isinstance(var, target.__class__):
            raise ValueError(
                "Variable for the VectorDifference inconsistent with the given target."
            )
        if not var.dof() == target.dof():
            raise ValueError(
                "Variable and target in the VectorDifference must have identical dof."
            )

        self.var = var
        self.target = target

        # register variable and target
        self.register_optim_vars(["var"])
        self.register_aux_vars(["target"])

<h2>Implement abstract functions</h2> 

Next, we need to implement the abstract functions of `CostFunction`: `dim`, `error`, `jacobians`, and `_copy_impl`:
- `dim`: returns the degrees of freedom (`dof`) of the error; in this case, this is the `dof` of the optimization variable `var`
- `error`: returns the difference of Vectors i.e. `var` - `target`
- `jacobian`: returns the Jacobian of the error with respect to the `var`
- `_copy_impl`: creates a deep copy of the internal class members

We illustrate these below (including once again the `__init__` function from above, so the class is fully defined.)

In [2]:
import torch 

class VectorDifference(th.CostFunction):
    def __init__(
        self,
        cost_weight: th.CostWeight,
        var: th.Vector,
        target: th.Vector,
        name: Optional[str] = None,
    ):
        super().__init__(cost_weight, name=name) 
        self.var = var
        self.target = target
        # to improve readability, we have skipped the data checks from code block above
        self.register_optim_vars(["var"])
        self.register_aux_vars(["target"])

    def error(self) -> torch.Tensor:
        return (self.var - self.target).tensor

    def jacobians(self) -> Tuple[List[torch.Tensor], torch.Tensor]:
        return [
            # jacobian of error function wrt var is identity matrix I
            torch.eye(self.dim(), dtype=self.var.dtype)  
            # repeat jacobian across each element in the batch
            .repeat(self.var.shape[0], 1, 1)  
            # send to variable device
            .to(self.var.device)  
        ], self.error()

    def dim(self) -> int:
        return self.var.dof()

    def _copy_impl(self, new_name: Optional[str] = None) -> "VectorDifference":
        return VectorDifference(  # type: ignore
            self.var.copy(), self.weight.copy(), self.target.copy(), name=new_name
        )

# Explanation of the Above

Why is the Jacobian like this? We have `var` and `target` but let's rewrite this in math notation to make it simpler. First, Jacobians need to operate on vector-valued functions, here we'll consider 2D examples, taking in a 2D vector and producing a 2D output. What is a bit confusing is that our function $f$ is taking in TWO 2D vectors, but we'll only differentiate with respect to one of them. Let's write $f$ as a function of two vectors as follows, where the $x$s indicate the `var` from above, and $y$s indicate the `target`.
$$
f( \langle x_1, x_2 \rangle, \langle y_1, y_2 \rangle) = \langle x_1 - y_1, x_2 - y_2 \rangle
$$

Technically, we have $f : (\mathbb{R}^2 \times \mathbb{R}^2) \to \mathbb{R}^2$. But if we want the derivative with respect to one vector then I think we can consider that to be the Jacobian which is suggested by the comments:

$$
J = \frac{\partial f}{\partial \langle x_1, x_2 \rangle} = \frac{\partial f}{\partial \langle x_1, x_2 \rangle} \langle x_1 - y_1, x_2 - y_2 \rangle
$$

The first row is: $\langle \frac{\partial f_1}{\partial x_1}, \frac{\partial f_1}{\partial x_2} \rangle = \langle \frac{\partial}{\partial x_1} (x_1 - y_1), \frac{\partial}{\partial x_2} (x_1 - y_1) \rangle = \langle 1, 0 \rangle$

The second row is: $\langle \frac{\partial f_2}{\partial x_1} \frac{\partial f_2}{\partial x_2} \rangle  = \langle \frac{\partial}{\partial x_1}(x_2-y_2), \frac{\partial}{\partial x_2} (x_2-y_2) \rangle = \langle 0, 1 \rangle$

Is that the way to think about it? The easiest way to adjust would be if there are some scalars we can use to multiply stuff, and ideally those are also in the Jacobian, so for example in the `error` function if we do `2. * self.var` then the Jacobian would be basically $2 \cdot I$. I think the reason why I didn't notice this earlier when I was fiddling around with Tutorial 00 is that `cf.jacobians` uses the internal Jacobian computed from the code and that is something fixed (and in any case we were not modifying the internal `error` function in that tutorial).

I am not sure if there is an automated way to check if the Jacobian is correctly computed. Edit: I should have checked, I am sure this will do it: https://github.com/facebookresearch/theseus/issues/323

<h2>Usage</h2>

We show now that the `VectorDifference` cost function works as expected. 

For this, we create a set of `VectorDifference` cost functions each over a pair of `Vector`s <i>a_i</i> and <i>b_i</i>, and add them to an `Objective`. We then create the data for each `Vector` <i>a_i</i> and <i>b_i</i> of the `VectorDifference` cost functions, and `update` the `Objective` with it. The code snippet below shows that the `Objective` error is correctly computed.

We use a `ScaleCostWeight` as the input `CostWeight` here: this is a scalar real-valued `CostWeight` used to weight the `CostFunction`; for simplicity we use a fixed value of 1. in this example.


Daniel: for some reason now `objective.error_squared_norm()` is not fixing things?

In [9]:
cost_weight = th.ScaleCostWeight(1.0)

# construct cost functions and add to objective
objective = th.Objective()
num_test_fns = 10
for i in range(num_test_fns):
    a = th.Vector(2, name=f"a_{i}")
    b = th.Vector(2, name=f"b_{i}")
    cost_fn = VectorDifference(cost_weight, a, b)
    objective.add(cost_fn)
    
# create data for adding to the objective
theseus_inputs = {}
for i in range(num_test_fns):
    # each pair of var/target has a difference of [1, 1]
    theseus_inputs.update({f"a_{i}": torch.ones((1,2)), f"b_{i}": 2 * torch.ones((1,2))})

objective.update(theseus_inputs)
# sum of squares of errors [1, 1] for 10 cost fns: the result should be 20
error_sq = objective.error_squared_norm()  # Daniel the usual fix.
print(f"Sample error squared norm: {error_sq.item()}")

ValueError: expected type 'Vector' or 'Tensor', got int

# Checking Jacobians

(New section added by Daniel Seita)

Let's try and modify the cost functions and then check if the Jacobians will work as expected.

Edit: wait that function was added on March 08, 2023. Yet from PyPI, 0.1.4 (which is what I am using) is from January 19, 2023. https://pypi.org/project/theseus-ai/#history  We need to update.

In [4]:
from theseus.utils.utils import check_jacobians

check_jacobians(cf=objective, verbose=True)

ImportError: cannot import name 'check_jacobians' from 'theseus.utils.utils' (/home/seita/miniconda3/envs/theseus/lib/python3.8/site-packages/theseus/utils/utils.py)